In [ ]:
%%capture
!mkdir sparse_vector

!pip install einops
!pip install torchvision
!pip install transformers==4.33.0
!pip install genomic-benchmarks
!pip install OmegaConf
!FLASH_ATTENTION_SKIP_CUDA_BUILD=TRUE pip install flash-attn --no-build-isolation
!pip install mamba_ssm==1.2.2

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import torch
from torch.utils import data
import random
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split, StratifiedKFold
from collections import Counter
import pandas as pd
import numpy as np
import scipy
from tqdm import trange
from tqdm import tqdm
from datetime import datetime
import sys
import os
import seaborn as sns
from matplotlib import pyplot as plt
from joblib import Parallel, delayed, dump, load
from matplotlib import pyplot as plt
from sparse_vector.sparse_vector import SparseVector
from scipy.signal import convolve2d, convolve
import time
from torch import nn
import torch.nn.functional as F
from sklearn.metrics import roc_auc_score, f1_score
from IPython.display import clear_output

import torch
from transformers import BertModel, AutoModel, BertConfig, PreTrainedTokenizer, AutoTokenizer, BertForTokenClassification, AutoModelForSequenceClassification
import collections

from transformers import utils

from torch.utils.data import DataLoader
import sklearn
from sklearn.metrics import accuracy_score
from torch.nn import CrossEntropyLoss

import gc
import torch
import math
import torch
import torch.nn as nn
import torch.nn.functional as F
from functools import partial
from einops import rearrange
from typing import Optional
from functools import partial
from torch import Tensor
from torchvision.ops import StochasticDepth
from collections import namedtuple

import json
import os
import subprocess
import transformers
from transformers import PreTrainedModel, AutoModelForCausalLM, PretrainedConfig
import importlib


import warnings
warnings.filterwarnings("ignore")

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("kuleshov-group/caduceus-ps_seqlen-131k_d_model-256_n_layer-16", trust_remote_code=True)
model = AutoModel.from_pretrained("kuleshov-group/caduceus-ps_seqlen-131k_d_model-256_n_layer-16", trust_remote_code=True)
cad_module_name = model.__class__.__module__
cls = getattr(importlib.import_module(cad_module_name), 'CaduceusForSequenceClassification')
model = cls.from_pretrained("kuleshov-group/caduceus-ps_seqlen-131k_d_model-256_n_layer-16", num_labels=2)

tokenizer_config.json:   0%|          | 0.00/1.49k [00:00<?, ?B/s]

tokenization_caduceus.py:   0%|          | 0.00/4.97k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/kuleshov-group/caduceus-ps_seqlen-131k_d_model-256_n_layer-16:
- tokenization_caduceus.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


special_tokens_map.json:   0%|          | 0.00/173 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.38k [00:00<?, ?B/s]

...

modeling_caduceus.py:   0%|          | 0.00/26.6k [00:00<?, ?B/s]

modeling_rcps.py:   0%|          | 0.00/9.80k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/kuleshov-group/caduceus-ps_seqlen-131k_d_model-256_n_layer-16:
- modeling_rcps.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.
A new version of the following files was downloaded from https://huggingface.co/kuleshov-group/caduceus-ps_seqlen-131k_d_model-256_n_layer-16:
- modeling_caduceus.py
- modeling_rcps.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


model.safetensors:   0%|          | 0.00/30.9M [00:00<?, ?B/s]

Some weights of CaduceusForSequenceClassification were not initialized from the model checkpoint at kuleshov-group/caduceus-ps_seqlen-131k_d_model-256_n_layer-16 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
def seq2kmer(seq, k):
    """
    Convert original sequence to kmers

    Arguments:
    seq -- str, original sequence.
    k -- int, kmer of length k specified.

    Returns:
    kmers -- str, kmers separated by space
    """
    kmer = [seq[x:x+k] for x in range(len(seq)+1-k)]
    kmers = " ".join(kmer)
    return kmers

In [ ]:
class PredDataset(data.Dataset):
    def __init__(self, chroms, dna_source, intervals, tokenizer):

        self.chroms = chroms
        self.intervals = intervals
        self.tokenizer = tokenizer
        self.dna_source = dna_source


    def __len__(self):
        return len(self.intervals)

    def __getitem__(self, index):
        interval = self.intervals[index]
        chrom = interval[0]
        begin = interval[1]
        end = interval[2]

        k_mers = seq2kmer(self.dna_source[chrom][begin:end+5].upper(),6)
        encoded_k_mers = self.tokenizer.encode_plus(k_mers, add_special_tokens=False, max_length=512)["input_ids"]

        return torch.LongTensor(encoded_k_mers), (chrom, begin, end)

In [ ]:
ASSEMBLY = "G4_cut_small"
chroms = [f'chr{i}' for i in list(range(1, 23)) + ['X']]
G4DNA = load(f'{ASSEMBLY}.pkl')

In [ ]:
def chrom_reader(chrom):
    files = sorted([i for i in os.listdir(f'/content/drive/My Drive/DeepZ_data_creation/data/hg19_dna/') if f"{chrom}_" in i])
    return ''.join([load(f"/content/drive/My Drive/DeepZ_data_creation/data/hg19_dna/{file}") for file in files])


DNA = {chrom:chrom_reader(chrom) for chrom in tqdm(chroms)}

100%|██████████| 23/23 [01:59<00:00,  5.19s/it]


In [ ]:
G4_kouzine = {}
for chrom in DNA:
    G4_kouzine[chrom] = np.zeros(len(DNA[chrom]), dtype = bool)


with open("/content/EndoQuad_hg19.bed")as f:
    for idx, line in enumerate(f):
        if idx>0:
            chrom, start, end, _ , _ , _ = line.strip().split()
            if chrom in G4_kouzine:
                G4_kouzine[chrom][int(start):int(end)] = 1

dump(G4_kouzine, 'endo.pkl')

['endo.pkl']

In [ ]:
G4 = load('endo.pkl')

In [ ]:
width = 512

np.random.seed(42)

ints_in = []
ints_out = []


for chrm in chroms:
    for st in trange(0, G4[chrm].shape[0] - width, width):
        interval = [st, min(st + width, G4[chrm].shape[0])]
        if G4[chrm][interval[0]: interval[1]].any():
            ints_in.append([chrm, int(interval[0]), int(interval[1]), 1])
        else:
            ints_out.append([chrm, int(interval[0]), int(interval[1]), 0])

print(len(ints_in))
print(len(ints_out))

ints_in_full = ints_in
ints_out_full = ints_out

100%|██████████| 303262/303262 [00:01<00:00, 280563.80it/s]

184812
5745459


In [ ]:
ints_in = ints_in_full
ints_out = [ints_out_full[i] for i in np.random.choice(range(len(ints_out_full)),
                                                    size=len(ints_in) * 2, replace=False)]
# ints_out = ints_out_full

print(len(ints_in))
print(len(ints_out))

184812
369624


In [ ]:
equalized = ints_in + ints_out

In [ ]:
divisions = list(StratifiedKFold(5, shuffle=True,
                                 random_state=42).split(equalized, [f"{elem[3]}_{elem[0]}"
                                         for i, elem
                                         in enumerate(equalized)]))

In [ ]:
dump([equalized, divisions], 'hg_divisions_g4.pkl', 3)

['hg_divisions_g4.pkl']

In [ ]:
width = 128
pad = 192
k_mer_pad = 5

def final_prediction(chrom):

    intervals = []
    ends = []


    prediction = np.zeros(len(DNA[chrom]), dtype=np.float32)


    for st in range(0, len(DNA[chrom]) - 512, width):
        interval = [st, min(st + 512, len(DNA[chrom]))]
        intervals.append([chrom, interval[0], interval[1]])

    pred_dataset = PredDataset(chroms, DNA, intervals,
                               tokenizer)

    params = {'batch_size':32, 'num_workers':5, 'shuffle':False}
    load_predict = data.DataLoader(pred_dataset, **params)


    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    print("Using device:", device)

    model.to(device)
    with torch.no_grad():
        for input_ids, intervals in tqdm(load_predict):
            input_ids = input_ids.to(device)
            outputs = model(input_ids)["logits"]
            # print(outputs.shape)
            pred = outputs[:, 1].cpu().numpy()
            # outputs = torch.softmax(model(input_ids = input_ids)['logits'],axis = -1).cpu().numpy()[:,:,1]
            for ind, interval in enumerate(zip(intervals[0], intervals[1], intervals[2])):
                prediction[interval[1]:interval[2]] = pred[ind]
                # if interval[1] == 0:

    dump(prediction, f'pred_caduceus_{chrom}', 3)

In [ ]:
device = "cuda"
torch.cuda.empty_cache()
# model.to(device)
# model.eval()

In [ ]:
for chrom in chroms[:2]:
    print(f"BEGIN CHROM {chrom}")
    final_prediction(chrom)

BEGIN CHROM chr1
Using device: cuda


  0%|          | 0/60853 [00:00<?, ?it/s]Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy.

KeyboardInterrupt: 

In [ ]:
equalized, divisions = load('hg_divisions_g4.pkl')

In [ ]:
com_len = sum([len(DNA[chrom]) for chrom in chroms])
sums = []

for chrom in tqdm(chroms):
    loc_sum = []
    # for model_num in range(5):
    vec = load(f"pred_caduceus_{chrom}")
    loc_sum.append(vec.sum())
    sums.append(loc_sum)

multipliers = np.array(sums).sum(axis=0) / com_len

100%|██████████| 23/23 [00:18<00:00,  1.25it/s]


In [ ]:
for chrom in tqdm(chroms):
    vecs = np.array([load(f"pred_caduceus_{chrom}")
                     ])
    res_vec = (vecs / multipliers[:, None]) * multipliers.mean()
    mean_vec = res_vec.mean(axis=0)

    test_ints = []

    train_inds, test_inds = divisions
    train_intervals, test_intervals = [equalized[i] for i in train_inds], [equalized[i] for i in test_inds]
    test_ints.extend([(inter) for inter in test_intervals if inter[0] == chrom])

    for inters in test_ints:
        mean_vec[inters[1]: inters[2]] = res_vec[inters[1]: inters[2]]
    dump(mean_vec, f"pred_caduceus_{chrom}", 3)

In [ ]:
all_pred = []
all_true = []
for chrom in tqdm(chroms):
    all_pred.append(load(f"pred_caduceus_{chrom}"))
    all_true.append(G4DNA[chrom][:].astype(int))

100%|██████████| 23/23 [00:53<00:00,  2.33s/it]


In [ ]:
roc_auc_score(np.concatenate(all_true), np.concatenate(all_pred))

In [ ]:
print(sklearn.metrics.classification_report(np.concatenate(all_true), np.concatenate(all_pred)>0.5, digits=4))

              precision    recall  f1-score   support

           0     0.9998    0.9970    0.9984 2654081885
           1     0.0365    0.3707    0.0664    813333

    accuracy                         0.9968 2654895218
   macro avg     0.5181    0.6839    0.5324 2654895218
weighted avg     0.9995    0.9968    0.9981 2654895218



In [ ]:
from tabulate import tabulate
def to_fwf(df, fname):
    content = tabulate(df.values.tolist(), tablefmt="plain")
    open(fname, "w").write(content)

pd.DataFrame.to_fwf = to_fwf

In [ ]:
pchroms, starts, ends = [], [], []
model_confidence_threshold = 0.05
min_length = 6

chroms = [f'chr{i}' for i in list(range(1, 23)) + ['X']]

for chrom in tqdm(chroms):
    pred = load(f'pred_caduceus_{chrom}')
    labeled, max_label = scipy.ndimage.label(pred>model_confidence_threshold)
    for idx in range(1,max_label+1):
        where = np.where(labeled == idx)[0]
        start = where[0]
        end = where[-1] + 1

        if end-start>min_length:
            pchroms.append(chrom)
            starts.append(start)
            ends.append(end)
pd.DataFrame(list(zip(pchroms, starts, ends))).to_fwf(f'CADUCEUS_thr_{model_confidence_threshold}_minlen_{min_length}.bed')